In [ ]:
%config InlineBackend.figure_format = "retina"
import matplotlib.pyplot as plt


plt.rcParams["font.size"] = 14

# Computational Inference via NumPyro

We want try out what we learned about computational inference and conduct the inferences we did in the previous notebook now with the use of sampling methods. We will be using the NumPyro library, to which we will do a short introduction before digging into the inference part. 

## 1. Model specification in NumPyro

As before we want to estimate the probability of a coin landing heads based on a number of observed flips. 

We assume like before a Beta-distributed prior with parameters $\alpha=\beta=20$.

With that assumption, the model takes the form:

$$
    p(\theta) = \mathrm{Beta}(20, 20) \\
    y_i \sim \mathrm{Bernoulli}(\theta), \quad i = 1, \dots, N \\
    p(y|N, \theta) = \mathrm{Binomial(y,N, \theta)}
$$

In [ ]:
ALPHA = 20
BETA = 20

### Numpy

Let's start by building a probabilistic model as a Python function just using NumPy.

In [ ]:
import numpy as np

We can use `np.random` to sample from distributions, e.g., from the prior.

In [ ]:
np.random.beta(ALPHA, BETA)

In [ ]:
def model(n=1):
    theta = np.random.beta(ALPHA, BETA)  # prior
    y = np.random.binomial(1, theta, size=n)  # likelihood
    return theta, y

Or indeed from the model. We simply sample theta, then n samples of y given that theta and return them.

In [ ]:
model(20)

### Numpyro

The corresponding model in NumPyro will look very similar, but instead of using distributions from `numpy.random` we use `numpyro.sample` statements and pass the corresponding distribution.

In [ ]:
import numpyro
import numpyro.distributions as dist

# tell numpyro to use multiple cores
numpyro.set_host_device_count(4)

In [ ]:
def model(n=1):
    theta = numpyro.sample("theta", dist.Beta(ALPHA, BETA))  # prior
    y = numpyro.sample("y", dist.Bernoulli(theta).expand((n,)))  # likelihood
    return theta, y

One final change is that NumPyro does not maintain global random state, so we need to manually seed the model. Don't worry about why this is here.

In [ ]:
from jax import random

rng_key = random.PRNGKey(42)

with numpyro.handlers.seed(rng_seed=rng_key):
    theta, y = model(n=20)

theta, y

### Improving the model

We can make a few improvements to the model.

- We will allow the user to pass in y values for the model to condition on. This ability to condition on observed data is crucial in order to make inferences about the latent variables ( $\theta$ in this case).
- We will replace `.expand` with a `numpyro.plate` context manager. This NumPyro primitive is another way of specifying batch dimensions inspired by [plate notation](https://en.wikipedia.org/wiki/Plate_notation) and makes for more readable code + additional useful output in the model trace. It's a construct for annotating conditionally independent variables. Within a plate context manager, sample sites will be automatically broadcasted to the size of the plate. 

In [ ]:
def model(y=None):

    n = y.shape[0] if y is not None else 20  # if you are sampling the prior

    # prior
    theta = numpyro.sample("theta", dist.Beta(ALPHA, BETA))

    # likelihood
    with numpyro.plate("n", n):  # think of it as range in python
        numpyro.sample("y", dist.Bernoulli(theta), obs=y)

Numpyro lets us create neat visualizations of our probabilistic  model.

In [ ]:
numpyro.render_model(model)

### Sampling from the prior
NumPyro provides a convenient interface for drawing multiple samples from a model.

In [ ]:
from numpyro.infer import Predictive

In [ ]:
N_SAMPLES = 4_000

In [ ]:
prior = Predictive(model, num_samples=N_SAMPLES)

rng_key, subkey = random.split(rng_key)
prior_samples = prior(subkey)

prior_samples["theta"].shape, prior_samples["y"].shape

In [ ]:
prior_samples

In [ ]:
plt.hist(prior_samples["theta"], bins=50, label="Prior samples")
plt.xlabel("Theta")
plt.legend()

## 2. Bayesian inference via NumPyro

Once we've built the model inference is pretty straight-forward. Let's fit the model to the data of our lazy 3-coin toss trial. 

In [ ]:
# Our data described as 3 outcomes from Bernoulli trials
y = np.array([1, 1, 1])

In [ ]:
from numpyro.infer import MCMC, NUTS

In [ ]:
mcmc = MCMC(NUTS(model), num_warmup=500, num_samples=int(N_SAMPLES / 4), num_chains=4)

rng_key, subkey = random.split(rng_key)
mcmc.run(subkey, y=y)

NumPyro provides a summary of your MCMC sampling, including statistics on the produced samples and an MCMC diagnostic: `r_hat`. We won't go into the details of `r_hat` here but note that `r_hat=1` indicates that our chains have converged.

In [ ]:
mcmc.print_summary()

In [ ]:
posterior_samples = mcmc.get_samples()

In [ ]:
posterior_samples["theta"].shape

In [ ]:
posterior_samples["theta"]

### Plot prior and posterior histograms

In [ ]:
f, ax = plt.subplots(figsize=(8, 5))
ax.hist(np.array(posterior_samples["theta"]), alpha=0.5, bins=50, label="posterior")
ax.hist(np.array(prior_samples["theta"]), alpha=0.5, bins=50, label="prior")
plt.legend(loc="best")
plt.show()

If we compare these histograms to the true pdf-plots from the previous notebook we notice they look very similar.

### Posterior Statistics

#### A. Expecation value

**Example** Compute the Monte Carlo estimate of the expectation of $\theta$ under the posterior. For posterior samples $\theta_i$, $i=1, ..., N$, this estimate is defined by

$$\mathbb{E}\left[\theta \vert y\right] \approx \frac{1}{N}\sum_{i=1}^N \theta_i.$$

In [ ]:
mean = posterior_samples["theta"].mean()
print(mean)

#### B. Event Probabilities

Compute the Monte Carlo estimate for the probability of $\theta \in [\theta_1, \theta_2]$ under the posterior. For posterior samples $\theta_i$, $i=1, ..., N$, this estimate is defined by,

$$ \int_{\theta_1}^{\theta_2} p(\theta \vert y) \mathrm{d} \theta \approx \frac{ \#\{\theta_i \in [\theta_1, \theta_2]\} }{N}$$

**Exercise:** What's the estimated probability of the coin being fair with a tolerance of 0.01? 

$$\text{Probability of fair} = \int_{0.49}^{0.51} p(\theta | y) d\theta \approx \frac{ \#\{\theta_i \in [0.49, 0.51]\} }{N} $$

In [ ]:
tolerance = 0.01

In [ ]:
posterior_samples["theta"]

In [ ]:
proba_fair = ...
print(proba_fair)

#### C. Quantiles

For a given probability $P \in [0,1]$ the associated posterior quantile can be approximated via posterior samples $\theta_i$, $i=1, ..., N$ by

$$ \arg\max_x \left\{ \int_0^{x} \ p(\theta \vert y) d\theta \le P \right\} \approx \arg\max_x \left\{ \frac{1}{N}\sum_{i=1}^N \chi(\theta_i \in [0, x]) \le P \right\} $$

**Exercise:** What's the 5-th and 95-th percentile estimate?

(Tip: you can use ```np.quantile``` instead of trying to implement the  formula of the approximation.)

In [ ]:
perc_5 = ...
perc_95 = ...

In [ ]:
print(perc_5)
print(perc_95)